In [2]:
pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365681 sha256=a6bf2d2969fe3f9ed8a816ca8b61afb39c08e81c9dec40d4dbe2b51eaeb9363c
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


In [3]:
from mpi4py import MPI
from multiprocessing import Pool

In [4]:
def is_palindrome(word):
    return word == word[::-1]

In [5]:
def mpi_palindrome(word):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        # Dividir la palabra en partes para enviar a cada proceso
        word_parts = [word[i::size] for i in range(size)]
    else:
        word_parts = None

    # Distribuir las partes de la palabra a todos los procesos
    word_part = comm.scatter(word_parts, root=0)

    # Verificar si la parte de la palabra es un palíndromo
    result = is_palindrome(word_part)

    # Recopilar los resultados de cada proceso
    results = comm.gather(result, root=0)

    if rank == 0:
        # Comprobar si todos los resultados son verdaderos para determinar si la palabra es un palíndromo
        is_palindrome_word = all(results)
        print(f"La palabra '{word}' es un palíndromo: {is_palindrome_word}")


In [6]:
def multiprocessing_palindrome(word):
    pool = Pool()
    result = pool.apply(is_palindrome, (word,))
    pool.close()
    pool.join()
    print(f"La palabra '{word}' es un palíndromo: {result}")


In [7]:
if __name__ == "__main__":
    word = "reconocer"  # Palabra de ejemplo

    print("Resultado utilizando MPI:")
    mpi_palindrome(word)

    print("\nResultado utilizando multiprocessing:")
    multiprocessing_palindrome(word)

Resultado utilizando MPI:
La palabra 'reconocer' es un palíndromo: True

Resultado utilizando multiprocessing:
La palabra 'reconocer' es un palíndromo: True
